# Exploración Inicial del Dataset

## Introducción

Este proyecto utiliza el dataset **Online Retail II**, el cual recopila todas las transacciones realizadas entre el 1 de diciembre de 2009 y el 9 de diciembre de 2011 por una empresa de comercio minorista en línea con sede y registro en el Reino Unido. La compañía se especializa en la venta de artículos de regalo únicos para toda ocasión, y entre sus principales clientes se encuentran mayoristas y minoristas internacionales.

Este dataset fue extraído del repositorio online Kaggle y está disponible publicamente en este [enlace](https://www.kaggle.com/datasets/mashlyn/online-retail-ii-uci).

[Siguiente: Análisis Exploratorio de Datos (EDA) ➡️](02_ES_analisis_eda.ipynb)

## Objetivo de este proyecto

El objetivo principal de este análisis es **explorar, limpiar y preparar los datos** para responder preguntas clave sobre el comportamiento de compra, los productos más vendidos, los patrones de ventas por país y el perfil de los clientes, sentando así las bases para la **visualización y toma de decisiones comerciales**.

En esta primera etapa, utilizo la biblioteca **pandas** de Python para realizar una exploración inicial del dataset, limpieza y transformaciones básicas. Posteriormente, estas herramientas se complementarán con consultas en SQL y visualizaciones en Power BI para un análisis más profundo.

## Resumen ejecutivo 

- Dataset final limpio: **1,054,675 filas** y **15 columnas**.
- Se eliminaron filas con **Description nula** (~0.41%) y **Price = 0**; se retiraron registros administrativos/comisiones.
- **Customer ID** nulo se gestiona con `Customer Label = 'unknown'` (solo para agregados; no para KPIs por cliente).
- **InvoiceDate** convertido a tipo datetime; columnas derivadas: Year, Month, Day Name, Year_Month.
- Importe por línea: **LineTotal = Quantity * Price**.
- Listo para la EDA visual en el siguiente notebook.

## Preguntas claves de análisis

Estas son algunas preguntas que responderé analizando este dataset:

1. ¿Cuáles son los 10 productos con mayores ventas?
2. ¿Cuáles son los países con más ventas?
3. ¿Cómo han cambiado las ventas a lo largo del tiempo (por mes)?
4. ¿Quiénes son los clientes que más compran?
5. ¿Qué porcentaje de facturas corresponden a devoluciones?
6. ¿Cuál es el ticket promedio por factura?
7. ¿En qué días de la semana se vende más?
8. ¿Existen patrones de ventas por temporada/mes?
9. ¿Cuáles son los productos más devueltos?
10. ¿Qué productos suelen comprarse juntos?

La finalidad de este proyecto es entender mejor el comportamiento de ventas y el perfil de los clientes.

## Exploración descriptiva y limpieza del dataset

---
### 1. Primero, cargo los datos y muestro las primeras filas para entender qué información tenemos.

In [1]:
# Importar el módulo pathlib para gestionar ruta de archivos
from pathlib import Path

# Importar la librería pandas para análisis de datos
import pandas as pd

# Carpeta raíz y rutas relativas
BASE_DIR = Path.cwd().parent
csv_path = BASE_DIR / 'data' / 'online_retail_II.csv'

# Cargar del dataset en el entorno
df = pd.read_csv(csv_path)

# Visualizar las primeras 5 filas del DataFrame
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


Describo las columnas del dataset

- `Invoice`: Número de factura, identifica cada transacción (Si empieza con 'C', hace referencia a una devolución).
- `StockCode`: Código de producto.
- `Description`: Descripción del producto.
- `Quantity`: Cantidad de productos vendida (puede ser negativa si se trata de una devolución).
- `InvoiceDate`: Fecha y hora de emisión de la factura.
- `Price`: Precio de venta (en libras esterlinas).
- `Customer ID`: Identificador único del cliente.
- `Country`: País donde se efectuó la compra del producto.

---
### 2. Realizo una exploración más a profundidad para saber con cuantas filas cuenta el archivo y saber si tenemos valores nulos.

In [2]:
# Mostrar información del DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


Podemos observar que:

- El dataset está formado por **1,067,371 filas** y **8 columnas**.
- Tenemos 5 columnas de tipo object (texto), 1 de enteros y 2 de flotantes.
- La columna `InvoiceDate` es una columna de **tipo texto**, deberíamos convertirla a columna **tipo fecha** si deseamos realizar análisis temporales.
- Las columnas `Description` y `Customer ID` presentan **valores nulos**.

### 3. Limpio y estandarizo los valores de las columnas que tienen texto.

In [3]:
# Eliminar espacios en blanco antes o después para cada columna de tipo object
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.strip()
    
# Convertir a mayúsculas las columnas Invoice, StockCode y Description
upper_cols = ['Invoice', 'StockCode', 'Description']

for col in upper_cols:
    df[col] = df[col].str.upper()

df['Description'] = df['Description'].str.lstrip('*')

df.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


---
### 4. Visualizamos la cantidad exacta de valores nulos por cada columna en el dataset.

In [4]:
# Contar valores nulos por columna
df.isnull().sum()

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64

Para saber qué columnas presentan un problema para el análisis, visualizamos el porcentaje de valores nulos sobre el total de filas.

In [5]:
# Calcular porcentaje de valores nulos por columna
df.isnull().sum() / len(df) * 100

Invoice         0.000000
StockCode       0.000000
Description     0.410541
Quantity        0.000000
InvoiceDate     0.000000
Price           0.000000
Customer ID    22.766873
Country         0.000000
dtype: float64

- Se aprecia que la columna `Description` tiene un **0.41%** de **valores nulos**, por lo que podríamos proceder a eliminar las filas.
- Sin embargo, la columna `Customer ID` tiene un **22.77%** de **valores nulos**, quizas de ventas sin registro (invitados).
- En esta situación debemos encontrar otra manera de proceder con los valores de la columna `Customer ID`.
- Si borramos las filas con valores nulos de `Customer ID` podriamos **perder información importante** para el análisis.

---
### 5. Tratamos los valores nulos

Eliminamos las filas donde la columna `Description` es nula, ya que una venta sin descripción de producto no aporta información útil al análisis.

In [6]:
# Contar filas antes y después de eliminar valores nulos en 'Description'
rows_before = len(df)
df = df.dropna(subset=['Description'])
rows_after = len(df)

Verificamos como quedó el dataset luego de la operación.

In [7]:
print(f'El dataset tenía {rows_before} filas antes de la limpieza')
print(f'El dataset tiene {rows_after} filas después de la limpieza')

El dataset tenía 1067371 filas antes de la limpieza
El dataset tiene 1062989 filas después de la limpieza


Luego de la eliminación de valores nulos de la columna `Description`, el dataset pasó de tener **1,067,371 filas** a tener **1,062,989 filas**. En este proceso sólo se eliminó un total de **0.41%** de los valores totales del dataset (4,382 filas), lo cual no afecta para nada el análisis a realizar.

In [8]:
# Verificar valores nulos después de la limpieza
df.isnull().sum()

Invoice             0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
Price               0
Customer ID    238625
Country             0
dtype: int64

En este dataset, la columna `Customer ID` tiene muchos valores nulos, ya que no todos los compradores están registrados.
Si reemplazamos los valores nulos por **'unknown'**, la columna pasará de ser numérica (float) a tipo texto (object).
Esto podría limitar ciertos análisis numéricos o de integridad sobre los IDs originales.

Para mantener la buena práctica de no modificar la columna original, y así conservar la posibilidad de realizar análisis numéricos en el futuro, crearemos una nueva columna llamada `Customer Label`.
- Esta columna tendrá el `Customer ID` para clientes registrados y 'unknown' para ventas realizadas por clientes anónimos o no identificados.
- Así, podemos usar `Customer Label` para todos los análisis y agrupaciones por cliente, sin perder la información original ni cambiar su tipo de dato.

In [9]:
# Crear columna 'Customer Label' y rellenar nulos con 'unknown'
df['Customer Label'] = df['Customer ID'].fillna('unknown')

Ahora mediante la creación de la columna auxiliar `Customer Label` podemos indicar la **ausencia de código de cliente** mediante el valor **'unknown'** y agruparlos para el análisis.

In [10]:
# Verificar 'Customer Label' en filas con 'Customer ID' nulo
df[df['Customer ID'].isnull()][['Invoice', 'Customer ID', 'Customer Label']].head()

,Invoice,Customer ID,Customer Label
263,489464,NaN,unknown
283,489463,NaN,unknown
284,489467,NaN,unknown
577,489525,NaN,unknown
578,489525,NaN,unknown


Exploramos nuevamente el dataset para verificar la ausencia de valores nulos.

In [11]:
# Verificar ausencia de valores nulos después de crear 'Customer Label'
df.isnull().sum()

Invoice                0
StockCode              0
Description            0
Quantity               0
InvoiceDate            0
Price                  0
Customer ID       238625
Country                0
Customer Label         0
dtype: int64

Tras la transformación, podemos observar la nueva columna `Customer Label` que fue creada a partir de `Customer ID` no presenta valores nulos.
La columna `Customer ID` se mantiene de la misma manera para poder realizar análisis avanzado de ser requerido.

---
### 6. Eliminamos filas con valores de precio 0

Elimino los registros con **precio igual a cero**, ya que **no aportan información relevante** para el análisis comercial y ayudan a mantener un dataset más **limpio y enfocado**.

Primero visualizamos un pequeño DataFrame de los datos que serán eliminados, para verificar que el proceso se esté realizando correctamente


In [12]:
# Crear DataFrame con filas donde 'Price' es 0 y mostrar 20 ejemplos aleatorios
price_zero = df['Price'] == 0
df_price_zero = df[price_zero]
display(df_price_zero.sample(20, random_state=18))

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Customer Label
922483,571112,22637,DAMAGES,-65,2011-10-13 17:36:00,0.0,NaN,United Kingdom,unknown
1035578,579429,20711,LOST IN SPACE,-98,2011-11-29 13:04:00,0.0,NaN,United Kingdom,unknown
269559,515464,22126,GIVEN AWAY,-5000,2010-07-12 15:48:00,0.0,NaN,United Kingdom,unknown
248851,513450,20726,LUNCH BAG WOODLAND,1,2010-06-24 15:03:00,0.0,NaN,United Kingdom,unknown
580650,540978,84050,COUNTED,-310,2011-01-12 15:04:00,0.0,NaN,United Kingdom,unknown
692101,550948,17109D,ADJUSTMENT,14,2011-04-21 15:56:00,0.0,NaN,United Kingdom,unknown
320939,520681,22465,HANGING METAL STAR LANTERN,1,2010-08-27 15:28:00,0.0,NaN,United Kingdom,unknown
1005638,577263,47503H,FOUND,66,2011-11-18 12:31:00,0.0,NaN,United Kingdom,unknown
100429,498888,15056N,WEDDING CO RETURNS?,-700,2010-02-23 13:08:00,0.0,NaN,United Kingdom,unknown
248839,513450,22202,MILK PAN PINK RETROSPOT,1,2010-06-24 15:03:00,0.0,NaN,United Kingdom,unknown


Una vez que se ha verificado que los datos a eliminar son los correctos, procedemos con la **eliminación de estas filas con valor precio 0** del dataset principal.

In [13]:
# Eliminar filas con 'Price' igual a 0
df = df[df['Price'] != 0]

---
### 7. Transformamos la columna `InvoiceDate` de tipo texto a tipo fecha

Es importante realizar esta conversión de tipo para poder realizar **análisis temporales** (ventas por mes, año, día de la semana, etc.).

In [14]:
# Convertir 'InvoiceDate' a formato datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')

Revisamos nuevamente el tipo de datos de cada columna en el dataset para verificar si la conversión se realizó correctamente y si no hay filas con datos nulos.

In [15]:
# Verificar tipos de datos tras la conversión
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1061169 entries, 0 to 1067370
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   Invoice         1061169 non-null  object        
 1   StockCode       1061169 non-null  object        
 2   Description     1061169 non-null  object        
 3   Quantity        1061169 non-null  int64         
 4   InvoiceDate     1061169 non-null  datetime64[ns]
 5   Price           1061169 non-null  float64       
 6   Customer ID     824293 non-null   float64       
 7   Country         1061169 non-null  object        
 8   Customer Label  1061169 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 81.0+ MB


Como se puede observar en la tabla superior, ahora la columna `InvoiceDate` es una columna de tipo fecha (datetime64[ns]) y no contiene filas con valores nulos.

Adicionalmente crearemos columnas de **día (de la semana)**, **mes**, **año** y **año/mes** para facilitar el análisis de tiempo de las ventas:

In [16]:
# Crear nuevas columnas de año, mes, día de la semana, año-mes y hora a partir de 'InvoiceDate'
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['Day Name'] = df['InvoiceDate'].dt.day_name()
df['Year_Month'] = df['InvoiceDate'].dt.to_period('M').dt.to_timestamp()
df['Hour'] = df['InvoiceDate'].dt.strftime('%H:00')

De esta manera podremos realizar un análisis de ventas por cada año, su evolución cronológica, ver si existe alguna estacionalidad en las ventas o incluso saber en qué día de la semana se suele vender más.

---
### 8. Creamos una columna con el importe total por cada línea

Agregamos una columna `LineTotal` que representa el valor total de cada línea de la factura, calculado como la cantidad de productos multiplicada por el precio unitario. Esta columna es fundamental para cualquier análisis de ingresos, ventas por producto, país, cliente o período de tiempo.

In [17]:
df['LineTotal'] = df['Quantity'] * df['Price']

Verificamos la creación de la columna y si los cálculos son correctos:

In [18]:
# Verificar 5 primeras líneas
df[['Quantity', 'Price', 'LineTotal']].head()

,Quantity,Price,LineTotal
0,12,6.95,83.4
1,12,6.75,81.0
2,12,6.75,81.0
3,48,2.10,100.8
4,24,1.25,30.0


Con la creación de la columna `LineTotal` nuestro dataset está listo para análisis de ventas.

---
### 9. Limpieza y estandarización de valores en la columna de países

> **Nota de actualización 1:**  
> Durante el análisis exploratorio (siguiente notebook), detecté que el valor `'EIRE'` debía unificarse como `'Ireland'`. Por ello, incorporo esta corrección en el proceso de limpieza para asegurar la consistencia en los datos.

In [19]:
# Reemplazar 'EIRE' por 'Ireland'
df['Country'] = df['Country'].replace({'EIRE': 'Ireland'})

Revisamos ahora **valores únicos** en la columna `Country` por si existe algún otro valor a transformar:

In [20]:
# Imprimir valores únicos en 'Country'
print(df['Country'].unique())

['United Kingdom' 'France' 'USA' 'Belgium' 'Australia' 'Ireland' 'Germany'
 'Portugal' 'Japan' 'Denmark' 'Nigeria' 'Netherlands' 'Poland' 'Spain'
 'Channel Islands' 'Italy' 'Cyprus' 'Greece' 'Norway' 'Austria' 'Sweden'
 'United Arab Emirates' 'Finland' 'Switzerland' 'Unspecified' 'Malta'
 'Bahrain' 'RSA' 'Bermuda' 'Hong Kong' 'Singapore' 'Thailand' 'Israel'
 'Lithuania' 'West Indies' 'Lebanon' 'Korea' 'Brazil' 'Canada' 'Iceland'
 'Saudi Arabia' 'Czech Republic' 'European Community']


Al realizar esta exploración de valores únicos, se identifica valores de **Unspecified**, **European Community** y **West Indies** que reflejan una ausencia de especificidad geográfica.

Es necesario revisar qué **porcentaje del total** representan estos dos valores:

In [21]:
# Imprimir la frecuencia de cada valor en 'Country'
print(df['Country'].value_counts())

Country
United Kingdom          975156
Ireland                  17861
Germany                  17615
France                   14329
Netherlands               5135
Spain                     3810
Switzerland               3188
Belgium                   3122
Portugal                  2620
Australia                 1910
Channel Islands           1664
Italy                     1534
Norway                    1454
Sweden                    1364
Cyprus                    1176
Finland                   1049
Austria                    938
Denmark                    817
Unspecified                756
Greece                     663
Japan                      582
Poland                     535
USA                        535
United Arab Emirates       500
Israel                     371
Hong Kong                  364
Singapore                  346
Malta                      299
Iceland                    253
Canada                     228
Lithuania                  189
RSA                        168


La suma de los valores **Unspecified** y **European Community** no llegan siquiera a representar un **0.08% del total**, por lo cual se puede decidir cambiar su valor a **Other** o simplemente **eliminarlos del DataFrame**.

En el caso de **West Indies**, si bien su representación en el DataFrame es mucho menor, sería interesante dejar estos valores para mantener el área geográfica a la que hacen referencia, ya que los países que lo conforman no aparecen en la lista de valores.

Es necesario resaltar que existe otro valor poco descripitvo en la columna `Country` que es **RSA**, así que, de igual manera que se trató el valor **EIRE**, este será cambiado a **South Africa** para **facilitar la comprensión** al momento de visualizar los datos. 

In [22]:
# Eliminar los valores 'Unspecified' y 'European Community' del DataFrame
df = df[~df['Country'].isin(['Unspecified', 'European Community'])]

# Reemplazar los valores 'RSA' por 'South Africa'
df['Country'] = df['Country'].replace({'RSA': 'South Africa'})

Luego de realizar estar transformaciones, verificaré como quedaron los valores de la columna `Country`:

In [23]:
# Verificar la frecuencia de valores después del cambio
print(df['Country'].value_counts())

Country
United Kingdom          975156
Ireland                  17861
Germany                  17615
France                   14329
Netherlands               5135
Spain                     3810
Switzerland               3188
Belgium                   3122
Portugal                  2620
Australia                 1910
Channel Islands           1664
Italy                     1534
Norway                    1454
Sweden                    1364
Cyprus                    1176
Finland                   1049
Austria                    938
Denmark                    817
Greece                     663
Japan                      582
USA                        535
Poland                     535
United Arab Emirates       500
Israel                     371
Hong Kong                  364
Singapore                  346
Malta                      299
Iceland                    253
Canada                     228
Lithuania                  189
South Africa               168
Bahrain                    126


Luego de aplicar las transformaciones y correcciones necesarias, la columna `Country` ha quedado limpia y estandarizada, lo que facilitará los análisis posteriores de ventas por país.

---
### 10. Limpieza proactiva de las otras columnas del DataFrame

Tras identificar y corregir valores atípicos en la columna `Country`, es fundamental aplicar la misma rigurosidad al resto de las columnas del DataFrame. Procedo a explorar y analizar los valores presentes en otras variables clave, con el objetivo de detectar posibles inconsistencias, valores atípicos o errores de registro que puedan afectar la calidad del análisis posterior. Este enfoque proactivo busca asegurar que todo el dataset esté limpio y preparado para los siguientes pasos del análisis.

Revisaré las columnas `Invoice`, `StockCode` y `Description` para verificar inconsistencias.

Según la documentación del dataset, toda factura cuyo identificador en la columna `Invoice` comienza con la letra 'C' corresponde a una devolución. Por lo tanto, **los únicos registros que deberían presentar valores negativos en `LineTotal` son aquellos asociados a devoluciones**. Este criterio permite validar la consistencia de los datos: si existen órdenes con valores negativos de `LineTotal` en facturas que no sean devoluciones, podrían representar errores de registro que deben ser corregidos o investigados antes de continuar con el análisis.

In [24]:
# Verificar filas con precio negativo y factura que no empiece con 'C'
df_invalid = df[(df['LineTotal'] < 0) & (~df['Invoice'].astype(str).str.startswith('C'))]
print(f'Cantidad de celdas negativas inválidas: {len(df_invalid)}')
display(df_invalid)

Cantidad de celdas negativas inválidas: 5


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Customer Label,Year,Month,Day Name,Year_Month,Hour,LineTotal
179403,A506401,B,ADJUST BAD DEBT,1,2010-04-29 13:36:00,-53594.36,NaN,United Kingdom,unknown,2010,4,Thursday,2010-04-01,13:00,-53594.36
276274,A516228,B,ADJUST BAD DEBT,1,2010-07-19 11:24:00,-44031.79,NaN,United Kingdom,unknown,2010,7,Monday,2010-07-01,11:00,-44031.79
403472,A528059,B,ADJUST BAD DEBT,1,2010-10-20 12:04:00,-38925.87,NaN,United Kingdom,unknown,2010,10,Wednesday,2010-10-01,12:00,-38925.87
825444,A563186,B,ADJUST BAD DEBT,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom,unknown,2011,8,Friday,2011-08-01,14:00,-11062.06
825445,A563187,B,ADJUST BAD DEBT,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom,unknown,2011,8,Friday,2011-08-01,14:00,-11062.06


Al realizar la exploración de los valores negativos en `LineTotal` que no tienen una estructura de devolución, se observa que corresponden a una descripción del tipo **Adjust bad debt**. Estos casos representan ajustes contables internos y **no serán considerados** en los análisis comerciales de ventas o devoluciones, ya que son valores asociados a facturas que no corresponden a devoluciones tradicionales.

En muchos sistemas de ventas es común encontrar registros que **no corresponden a ventas reales de productos**, sino a ajustes contables, pruebas internas, muestras gratuitas u otros movimientos administrativos. Estos registros suelen identificarse en la columna `Description` por términos como **'bad debt', 'sample', 'post', 'manual', 'test', 'check', 'discount' o 'promotion'**.

Para asegurar que el análisis comercial refleje únicamente ventas y devoluciones reales, identifico y excluyo estos registros del dataset limpio.

In [25]:
# Lista de palabras clave que indican registros administrativos o especiales
keywords = ['bad debt', 'sample', 'post', 'manual', 'test', 'check', 'discount', 'promotion']

# Filtro booleano para filas con descripciones especiales
special_filter = df['Description'].str.contains('|'.join(keywords), case=False, na=False)

# Crear DataFrame con registros especiales
df_special = df[special_filter]

# Revisar el tamaño del DataFrame especial
print(f'Cantidad de celdas especiales en el DataFrame: {len(df_special)}')

# Tomar una muestra aleatoria de 20 elementos del DataFrame filtrado
display(df_special.sample(20, random_state=18))

Cantidad de celdas especiales en el DataFrame: 7295


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Customer Label,Year,Month,Day Name,Year_Month,Hour,LineTotal
354631,523893,22944,CHRISTMAS METAL POSTCARD WITH BELLS,12,2010-09-24 14:56:00,1.25,13381.0,United Kingdom,13381.0,2010,9,Friday,2010-09-01,14:00,15.00
178340,506285,M,MANUAL,3,2010-04-28 16:49:00,1.45,16767.0,United Kingdom,16767.0,2010,4,Wednesday,2010-04-01,16:00,4.35
574958,540519,POST,POSTAGE,2,2011-01-09 12:12:00,28.00,12793.0,Portugal,12793.0,2011,1,Sunday,2011-01-01,12:00,56.00
685313,C550365,POST,POSTAGE,-1,2011-04-18 11:31:00,11.00,12731.0,France,12731.0,2011,4,Monday,2011-04-01,11:00,-11.00
405775,528179,M,MANUAL,3,2010-10-21 10:49:00,0.85,14227.0,United Kingdom,14227.0,2010,10,Thursday,2010-10-01,10:00,2.55
306054,519238,21407,BROWN CHECK CAT DOORSTOP,6,2010-08-15 13:28:00,4.25,13405.0,United Kingdom,13405.0,2010,8,Sunday,2010-08-01,13:00,25.50
57087,494495,21407,BROWN CHECK CAT DOORSTOP,1,2010-01-14 17:43:00,8.47,NaN,United Kingdom,unknown,2010,1,Thursday,2010-01-01,17:00,8.47
796623,560651,M,MANUAL,1,2011-07-20 11:38:00,451.42,15802.0,United Kingdom,15802.0,2011,7,Wednesday,2011-07-01,11:00,451.42
396355,527392,M,MANUAL,1,2010-10-17 13:28:00,3.75,14044.0,United Kingdom,14044.0,2010,10,Sunday,2010-10-01,13:00,3.75
767908,C558347,S,SAMPLES,-1,2011-06-28 14:47:00,9.90,NaN,United Kingdom,unknown,2011,6,Tuesday,2011-06-01,14:00,-9.90


En la lista de valores de descripciones identificadas como celdas especiales se observa que hay algunos **falsos positivos**, como 'BROWN CHECK CAT DOORSTOP' o 'VICTORIAN METAL POSTCARD SPRING', los cuales representan **ventas reales** y no registros administrativos.

Por ello, es necesario revisar la lista de valores únicos de este DataFrame para identificar qué otros **productos legítimos** están siendo incluidos de manera incorrecta por el filtro y ajustar la lógica de exclusión si es necesario.

In [26]:
# Contar los valores especiales
df_special['Description'].value_counts()

Description
POSTAGE                                2112
DOTCOM POSTAGE                         1439
MANUAL                                 1418
BROWN CHECK CAT DOORSTOP                472
VICTORIAN  METAL POSTCARD SPRING        434
SET OF 12  VINTAGE POSTCARD SET         263
DISCOUNT                                177
CHRISTMAS METAL POSTCARD WITH BELLS     172
POSTE FRANCE CUSHION COVER              155
SUNSET CHECK HAMMOCK                    115
SAMPLES                                 104
PAIR PADDED HANGERS PINK CHECK          100
WRAP ALPHABET POSTER                     64
VINTAGE POST OFFICE CABINET              54
VICTORIAN METAL POSTCARD CHRISTMAS       49
YELLOW EASTER EGG HUNT START POST        49
GREEN EASTER EGG HUNT START POST         29
BLUE EASTER EGG HUNT START POST          27
MULTICOLOUR CRUSOE CHECK LAMPSHADE       17
THIS IS A TEST PRODUCT.                  14
BLUE CHECK BAG W HANDLE 34X20CM          11
BLUE CRUSOE CHECK LAMPSHADE               7
SET 10 CARDS PERFECT

Se puede corroborar que las palabras clave **"Check"** y **"Post"** están siendo incluidas erróneamente por el filtro actual, ya que los resultados para **"Check"** corresponden a productos reales. A partir de este análisis, se puede inducir que la descripción que realmente debe filtrarse es **"Postage"**. Usaré estos hallazgos para actualizar y refinar el filtro de limpieza.

In [27]:
# Actaulizar la lista de palabras clave que indican registros administrativos o especiales
keywords = ['bad debt', 'sample', 'postage', 'manual', 'test', 'discount', 'promotion']

# Refinar el filtro para filas con descripciones especiales
revised_filter = df['Description'].str.contains('|'.join(keywords), case=False, na=False)

# Actualizar del DataFrame con registros especiales
df_special = df[revised_filter]

# Visualizar de los valores únicos del DataFrame con registros especiales
display(df_special['Description'].value_counts())

Description
POSTAGE                    2112
DOTCOM POSTAGE             1439
MANUAL                     1418
DISCOUNT                    177
SAMPLES                     104
THIS IS A TEST PRODUCT.      14
ADJUST BAD DEBT               6
Name: count, dtype: int64

Luego de afinar el filtro y revisar los resultados, confirmo que ya **no se incluyen valores erróneos** de forma accidental. Por lo tanto, puedo proceder con la **eliminación segura** de estos registros del dataset.

In [28]:
# Eliminar valores especiales del DataFrame
df = df[~revised_filter]

Los registros administrativos o especiales identificados por el filtro refinado se eliminan del dataset, asegurando que el análisis posterior se base únicamente en **ventas y devoluciones reales**.

Revisaré la columna `Description` para verificar inconsistencias en las descripciones.

In [29]:
# Verificar los valores de la columna 'Description'
print("Cantidad de productos únicos:", df['Description'].nunique())
print("\nTop 20 productos más frecuentes:\n", df['Description'].value_counts().head(20))
print("\nEjemplos aleatorios de descripciones:\n", df['Description'].sample(20, random_state=18))

Cantidad de productos únicos: 5365

Top 20 productos más frecuentes:
 Description
WHITE HANGING HEART T-LIGHT HOLDER    5912
REGENCY CAKESTAND 3 TIER              4404
JUMBO BAG RED RETROSPOT               3465
ASSORTED COLOUR BIRD ORNAMENT         2954
PARTY BUNTING                         2763
STRAWBERRY CERAMIC TRINKET BOX        2608
LUNCH BAG  BLACK SKULL.               2528
JUMBO STORAGE BAG SUKI                2431
HEART OF WICKER SMALL                 2313
JUMBO SHOPPER VINTAGE RED PAISLEY     2295
60 TEATIME FAIRY CAKE CASES           2266
PAPER CHAIN KIT 50'S CHRISTMAS        2215
LUNCH BAG SPACEBOY DESIGN             2204
REX CASH+CARRY JUMBO SHOPPER          2196
HOME BUILDING BLOCK WORD              2192
WOODEN FRAME ANTIQUE WHITE            2189
LUNCH BAG CARS BLUE                   2183
NATURAL SLATE HEART CHALKBOARD        2150
BAKING SET 9 PIECE RETROSPOT          2146
WOODEN PICTURE FRAME WHITE FINISH     2114
Name: count, dtype: int64

Ejemplos aleatorios de descripc

Y de la misma manera con la columna `StockCode`.

In [30]:
# Verificar los valores de la columna 'StockCode'
print("Cantidad de códigos de producto únicos:", df['StockCode'].nunique())
print("\nTop 20 códigos de producto más frecuentes:\n", df['StockCode'].value_counts().head(20))
print("\nEjemplos aleatorios de códigos de producto:\n", df['StockCode'].sample(20, random_state=18))

Cantidad de códigos de producto únicos: 4752

Top 20 códigos de producto más frecuentes:
 StockCode
85123A    5921
22423     4404
85099B    4235
21212     3315
20725     3255
84879     2954
47566     2763
21232     2742
22383     2539
22197     2538
20727     2528
21931     2431
22386     2347
22469     2313
22411     2295
84991     2266
22382     2249
22384     2226
21080     2223
22086     2215
Name: count, dtype: int64

Ejemplos aleatorios de códigos de producto:
 1006158     23296
264977     84970L
836688      22382
871276     84971S
927174     82494L
991023      22138
384228      22271
814995      23049
826389      21080
354560      22582
293768      21928
742701      21933
1015035     23169
222051      21213
993519      20725
761952      23002
490880      22749
1005487     22083
398088      22740
50709       21680
Name: StockCode, dtype: object


Al revisar estas columnas no se encuentra **ningún valor atípico** por lo cual se continuará con el proceso de limpieza del dataset.

> **Nota de actualización 2:**  
> Durante el análisis exploratorio (siguiente notebook), detecté que existían valores como **"AMAZON FEE"** y **"Bank Charges"** que **no representan ventas reales**, por lo que se procederá a eliminar estos valores en el proceso de limpieza. Estos valores pueden ser facilmente identificados por sus valores "AMAZONFEE" y "BANKCHARGES" en la columna `StockCode`.

> **Nota de actualización 3:**  
> Durante el análisis exploratorio (siguiente notebook), detecté adicionalmente que existían valores de ajuste bajo en la columna `StockCode` con las etiquetas **"ADJUST"** y **"ADJUST2"**. De igual forma se eliminarán estos valores del DataFrame.

> **Nota de actualización 4:**  
> Durante la fase 3 del proyecto (Dashboard en Power BI), en el proceso de validación visual de los datos, se identificó la presencia del valor **"CRUK"** en la columna `StockCode`, asociado a la descripción **"CRUK commission"**, así como el valor **"23444"**, correspondiente a pagos adicionales por **envío urgente de productos al siguiente día útil** y el valor **"PADS"**, con un valor simbólico de 0.001. Estos registros **no representan transacciones de venta ni de devolución**, sino cargos administrativos, logísticos o comisiones internas. Con el fin de preservar la integridad del análisis, estos valores serán eliminados del DataFrame y se documenta el criterio para su aplicación en futuras actualizaciones del dataset.

In [31]:
# Eliminar filas que no representen ventas reales
df = df[~df['StockCode'].isin(['AMAZONFEE', 'BANK CHARGES', 'ADJUST', 'ADJUST2', 'CRUK', '23444', 'PADS'])]

Procedo a revisar si los valores fueron correctamente eliminados

In [32]:
# Verificar si las filas con estos valores fueron eliminadas
suspects = df[df['StockCode'].isin(['AMAZONFEE', 'BANK CHARGES','ADJUST', 'ADJUST2', 'CRUK', '23444', 'PADS'])]
display(suspects)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Customer Label,Year,Month,Day Name,Year_Month,Hour,LineTotal


Como se puede observar, es un DataFrame vacío, lo que indica que la eliminación procedió exitosamente.

> **Nota de actualización 5:**  
> Durante el análisis exploratorio detecté que algunos registros estaban etiquetados como **"Gift Voucher"**, los cuales representan la venta de vales o tarjetas de regalo y **no corresponden a una venta directa de productos físicos o servicios consumidos en el momento**. Tras una reflexión profunda, decidí eliminar estos registros del dataset para evitar inflar artificialmente las métricas de ventas reales, ya que estas gift vouchers representan un compromiso futuro y no ingreso efectivo inmediato. Esto es especialmente relevante en análisis de ventas e-commerce donde el foco está en medir el consumo efectivo.

In [33]:
# Contar filas antes y después de eliminar valores con descripción 'Gift Voucher'
print('Filas antes de la limpieza: ', len(df))
print('Filas que contenian valores "gift voucher" limpiadas: ', df['Description'].str.contains('gift voucher', case=False, na=False).sum())
df = df[~df['Description'].str.contains('gift voucher', case=False, na=False)]

print('Filas después de la limpieza: ', len(df))

Filas antes de la limpieza:  1054754
Filas que contenian valores "gift voucher" limpiadas:  79
Filas después de la limpieza:  1054675


> **Nota de actualización 6:**  
>  Durante la fase 3 del proyecto (Dashboard en Power BI), me di cuenta que algunos valores en la columna **`StockCode`** estaban asociados con multiples entradas en la columna **`Description`**. Esto se debería a errores durante el ingreso de datos al dataset. Procederé a normalizar estas descripciones con el **valor más frecuente** para cada `StockCode`.

In [34]:
# Crear una copia para evitar advertencias
df = df.copy()

# Mantener la columna 'Description' original para trazabilidad
df['Description_raw'] = df['Description']

# Normalizar valores en la columna 'Description_raw'
desc_norm = (
    df['Description_raw'].str.upper().str.strip().str.replace(r'\s+', ' ', regex=True)
)

# Calcular la Description más frecuente por cada StockCode
mode_map = (
    pd.concat([df['StockCode'], desc_norm], axis=1)
      .groupby('StockCode', as_index=True)['Description_raw']
      .agg(lambda s: s.value_counts(dropna=True).idxmax())
)

# Crear una columna Description limpia y reemplazar la original
df['Description_clean'] = df['StockCode'].map(mode_map)
df['Description'] = df['Description_clean']

# Revisar cuántos valores únicos fueron reducidos
before = df['Description_raw'].nunique(dropna=True)
after  = df['Description'].nunique(dropna=True)

print(f"Descripciones únicas antes:  {before:,}")
print(f"Descripciones únicas después: {after:,}")
print(f"Reducidos: {before - after:,}")

# Mostrar una tabla con StockCodes conflictivos y Descriptions
dup_check = (df.groupby('StockCode')['Description_raw']
               .nunique(dropna=True)
               .rename('n_desc'))
conflict_codes = dup_check[dup_check > 1].index

df_conflicts = (
    df.loc[df['StockCode'].isin(conflict_codes),
           ['StockCode','Description_raw','Description']]
      .drop_duplicates()
      .sort_values(['StockCode','Description_raw'])
)

# Revisar los primeros 30 conflictos resueltos
display(df_conflicts.head(30))

print(f'Número de StockCodes distintos: {df['StockCode'].nunique()}')
print(f'Número de Descriptions distintos: {df['Description'].nunique()}')


Descripciones únicas antes:  5,348
Descripciones únicas después: 4,697
Reducidos: 651


,StockCode,Description_raw,Description
548898,15058A,BLUE POLKADOT GARDEN PARASOL,BLUE POLKADOT GARDEN PARASOL
67142,15058A,BLUE WHITE SPOTS GARDEN PARASOL,BLUE POLKADOT GARDEN PARASOL
461044,15058B,PINK POLKADOT GARDEN PARASOL,PINK POLKADOT GARDEN PARASOL
54894,15058B,PINK WHITE SPOTS GARDEN PARASOL,PINK POLKADOT GARDEN PARASOL
314800,16012,FOOD/DRINK SPONGE STICKERS,FOOD/DRINK SPONGE STICKERS
4157,16012,FOOD/DRINK SPUNGE STICKERS,FOOD/DRINK SPONGE STICKERS
181581,16151A,FLOWER DES BLUE HANDBAG/ORANG HANDL,FLOWERS HANDBAG BLUE AND ORANGE
710756,16151A,FLOWERS HANDBAG BLUE AND ORANGE,FLOWERS HANDBAG BLUE AND ORANGE
663541,16156L,WRAP CAROUSEL,"WRAP, CAROUSEL"
49282,16156L,"WRAP, CAROUSEL","WRAP, CAROUSEL"


Número de StockCodes distintos: 4737
Número de Descriptions distintos: 4697


---
### 11. Ordenamos las columnas del dataset

Si bien no es necesario que las columnas estén ordenadas de una manera lógico para un análisis en pandas, es una buena práctica ordenar las columnas del dataset para una posterior exportación a Excel, SQL, o para realizar visualizaciones en PowerBI como haremos en los siguientes pasos

Para ello utilizaremos la siguiente fórmula para:
- Colocar la columna `LineTotal` al lado de `Price`.
- Colocar la columna `Customer Label` a la derecha de la columna `Customer ID` a la cual referencia.
- Colocar las columnas de fecha a continuación de `InvoiceDate`.

In [35]:
# Ordenar columnas del dataset
df.insert(df.columns.get_loc('Price') + 1, 'LineTotal', df.pop('LineTotal'))
df.insert(df.columns.get_loc('Customer ID') + 1, 'Customer Label', df.pop('Customer Label'))
df.insert(df.columns.get_loc('InvoiceDate') + 1, 'Hour', df.pop('Hour'))
df.insert(df.columns.get_loc('InvoiceDate') + 2, 'Day Name', df.pop('Day Name'))
df.insert(df.columns.get_loc('InvoiceDate') + 3, 'Month', df.pop('Month'))
df.insert(df.columns.get_loc('InvoiceDate') + 4, 'Year', df.pop('Year'))
df.insert(df.columns.get_loc('InvoiceDate') + 5, 'Year_Month', df.pop('Year_Month'))

Reseteamos el índice del dataset luego de finalizar la limpieza

In [36]:
# Restablecer índice del dataset
df.reset_index(drop=True, inplace=True)

Revisamos como quedó finalmente el orden de las columnas en nuestro dataset:

In [37]:
# Mostrar 5 primeras filas
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Hour,Day Name,Month,Year,Year_Month,Price,LineTotal,Customer ID,Customer Label,Country,Description_raw,Description_clean
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,07:00,Tuesday,12,2009,2009-12-01,6.95,83.4,13085.0,13085.0,United Kingdom,15CM CHRISTMAS GLASS BALL 20 LIGHTS,15CM CHRISTMAS GLASS BALL 20 LIGHTS
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,07:00,Tuesday,12,2009,2009-12-01,6.75,81.0,13085.0,13085.0,United Kingdom,PINK CHERRY LIGHTS,PINK CHERRY LIGHTS
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,07:00,Tuesday,12,2009,2009-12-01,6.75,81.0,13085.0,13085.0,United Kingdom,WHITE CHERRY LIGHTS,WHITE CHERRY LIGHTS
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,07:00,Tuesday,12,2009,2009-12-01,2.10,100.8,13085.0,13085.0,United Kingdom,"RECORD FRAME 7"" SINGLE SIZE","RECORD FRAME 7"" SINGLE SIZE"
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,07:00,Tuesday,12,2009,2009-12-01,1.25,30.0,13085.0,13085.0,United Kingdom,STRAWBERRY CERAMIC TRINKET BOX,STRAWBERRY CERAMIC TRINKET BOX


In [38]:
# Mostrar información del DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1054675 entries, 0 to 1054674
Data columns (total 17 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   Invoice            1054675 non-null  object        
 1   StockCode          1054675 non-null  object        
 2   Description        1054675 non-null  object        
 3   Quantity           1054675 non-null  int64         
 4   InvoiceDate        1054675 non-null  datetime64[ns]
 5   Hour               1054675 non-null  object        
 6   Day Name           1054675 non-null  object        
 7   Month              1054675 non-null  int32         
 8   Year               1054675 non-null  int32         
 9   Year_Month         1054675 non-null  datetime64[ns]
 10  Price              1054675 non-null  float64       
 11  LineTotal          1054675 non-null  float64       
 12  Customer ID        820166 non-null   float64       
 13  Customer Label     1054675 

Ahora tenemos el dataset ordenado y limpio para realizar consultas y análisis, además de estar listo para ser exportado y compartido con colaboradores.

---
### 12. Guardamos el dataset limpio en formato .csv

In [39]:
# Guardar dataset en formato .csv
csv_save_path = BASE_DIR / 'data' / 'online_retail_clean.csv'
df.to_csv(csv_save_path, index=False)

Guardar el dataset limpio en un archivo independiente es una buena práctica en cualquier proyecto de análisis de datos. Esto permite:

- **Reproducibilidad**: Otros pueden retomar el proyecto y partir desde un punto de datos ya depurado, sin tener que repetir la limpieza inicial.
- **Ahorro de tiempo**: Si el análisis o las visualizaciones requieren ser rehechos, es mucho más eficiente cargar el dataset limpio que procesar el archivo crudo cada vez.
- **Seguridad y control de versiones**: Mantener los datos originales separados de los datos limpios ayuda a evitar errores accidentales y facilita comparar resultados antes y después del proceso de limpieza.
- **Escalabilidad**: Permite que distintos miembros del equipo trabajen en análisis avanzados sin preocuparse por inconsistencias debidas a pasos de limpieza no replicados.

De esta manera, se garantiza un flujo de trabajo profesional y eficiente a lo largo de todo el proyecto.

---
## Conclusión:

- El dataset quedó **limpio de valores nulos** en `Description` mediante la eliminación de estas filas.
- Los valores nulos de `Customer ID` fueron gestionados mediante la creación de la **columna auxiliar** `Customer Label` y completando los valores nulos con el valor **'unknown'**.
- Se eliminaron los **valores con precio 0** ya que no aportaban al análisis general.
- La **columna** `InvoiceDate` fue transformada a **tipo fecha** para facilitar el **análisis temporal del dataset**.
- De la columna `InvoiceDate` extrajimos **cuatro columnas adicionales** (`Year`, `Month`, `Day Name` y `Year_Month`) que nos brindarán *insights* más precisos para un mejor análisis de tiempo.
- Se creó la **columna** `LineTotal` para visualizar el monto total de cada línea de factura.
- La columna `Country` fue estandarizada para asegurar una mejor **facilidad de lectura** y se eliminaron valores poco específicos.
- Se procedió a filtrar y eliminar los **registros administrativos** y las **comisiones varias** para asegurar que el dataset muestre **transacciones reales**.
- Las columnas fueron ordenadas de una manera coherente para su posterior **exportación y colaboración**.
- El **dataset final** cuenta con **15 columnas y 1,054,675 filas**.

[Siguiente: Análisis Exploratorio de Datos (EDA) ➡️](02_ES_analisis_eda.ipynb)